> imports

In [1]:
from matplotlib import pyplot as plt
import numpy as np

# Задание 1
 > минимизация квадратичной функции

In [52]:
#  Setup
example_A = np.array([[2, 1], [1, 2]])
example_b = np.array([[-2], [1]])
example_x = np.array([[1], [1], [1]])

A_task = np.array([
    [2, 0, 1],
    [0, 2, 0],
    [1, 0, 2]
])
b_task = np.array([
    [-2],
    [1],
    [-1]
])

epsilon = 1e-6

def get_e(size = 2):
    return np.array([[1], *[[0] for x in range(size - 1)]])


In [37]:
#  1.1
# a

# number
def f1(x_1, x_2):
    return x_1 ** 2 + x_1 * x_2 + x_2 ** 2 - 2 * x_1 + x_2

# vector

def makeF(A, b):
    return lambda x: ((x.T.dot(A).dot(x)) / 2 + x.T.dot(b))[0][0]

def F1(x):
    A = A_task
    b = b_task

    print('A =\n',  np.array_str(A, precision=1, suppress_small=True))
    print('b =\n', np.array_str(b, precision=1, suppress_small=True))

    print("A = A':", all( ( A == A.T).tolist() ) )
    va, vb = np.linalg.eigh(A)
    nums = sorted(va.tolist())
    print('Собственные числа: ', *nums)
    m, M = np.linalg.norm(x)**2 * nums[0], np.linalg.norm(x)**2 * nums[-1]

    print(
        f' {nums[0]}||x||^2 <= (x, Ax) <= {nums[1]}||x||^2: ', 
        all( [m <= x.T.dot(A).dot(x)[0][0], M >= x.T.dot(A).dot(x)[0][0]]) 
    )

F1(example_x)

A =
 [[2 0 1]
 [0 2 0]
 [1 0 2]]
b =
 [[-2]
 [ 1]
 [-1]]
A = A': True
Собственные числа:  1.0 2.0 3.0
 1.0||x||^2 <= (x, Ax) <= 2.0||x||^2:  True


In [67]:
# 1.2
def F2():
    A = A_task
    b = b_task
    
    F = makeF(A, b)
    print('A =\n',  np.array_str(A, precision=1, suppress_small=True))
    print('b =\n', np.array_str(b, precision=1, suppress_small=True))

    solution = np.linalg.solve(A, b)

    print('Solution for Ax + b = 0: ', solution.T.tolist()[0]) 
    print('F(x\u0304): ', F(solution))
F2()

A =
 [[2 0 1]
 [0 2 0]
 [1 0 2]]
b =
 [[-2]
 [ 1]
 [-1]]
Solution for Ax + b = 0:  [-1.0, 0.5, 0.0]
F(x̄):  3.75


In [90]:
# 1.3 Метод градиентного спуска
def F3(A, b, F, maxiter = 1000, log = False):
    # k = 0
    x_k = get_e(len(A))
    F_x_prev = 0
    F_x_k = F(x_k)
    q_k = A.dot(x_k) + b
    if(log):
        print(f'k = {0}: x_k = {x_k.T.tolist()[0]}')
    for k in range(1, maxiter):
        Aq_k = A.dot(q_k)
        d, dd = q_k.T.dot(q_k)[0][0], q_k.T.dot(Aq_k)[0][0]
        mu_k = -d / dd
        x_k = x_k + mu_k * q_k
        q_k = A.dot(x_k) + b
        F_x_prev = F_x_k
        F_x_k = F(x_k)
        if(log):
            print(f'k = {k}: x_k = {x_k.T.tolist()[0]}; F(x_k) = {F_x_k}')

        if(abs(np.linalg.norm(q_k)) < epsilon):
            if(log):
                print('q_K = 0. exiting')
            return (x_k, F_x_k)

        if( abs(F_x_k - F_x_prev) < epsilon):
            if(log):
                print('||F(x_k) - F(x_k+1)|| = 0. exiting')
            return (x_k, F_x_k)

    return (x_k, F_x_k)

A = A_task
b = b_task
F = makeF(A, b)
x_min, fx_min = F3(A, b, F, 1000, True)

# print(f'X_min: {x_min.T.tolist()[0]}, F(x_min): {fx_min}')


k = 0: x_k = [1, 0, 0]
k = 1: x_k = [1.0, -0.5, 0.0]; F(x_k) = -1.25
q_K = 0. exiting


In [91]:
# 1.4 Метод покоординатного спуска
def F4(A, b, F, maxiter = 1000, log = False):
    # масстив базисных векторов
    E = [
        np.array([
            *[[0] for j in range(i)],
            [1],
            *[[0] for j in range( len(A) - i - 1)]
        ])
        for i in range(len(A))
    ]

    x_k = np.array([[0] for _ in range(len(A))])
    F_x_k = F(x_k)
    for k in range(maxiter):
        e = E[k % len(A)]
        d, dd = e.T.dot( A.dot(x_k) + b )[0][0], e.T.dot( A.dot(e) )[0][0]
        mu_k = - d / dd
        x_k = x_k + mu_k * e

        F_x_prev = F_x_k
        F_x_k = F(x_k)
        if(log):
            print(f'k = {k}: x_k = {x_k.T.tolist()[0]}; F(x_k) = {F_x_k}')
        if( abs(F_x_k - F_x_prev) < epsilon):
            if(log):
                print('||F(x_k) - F(x_k+1)|| = 0. exiting')
            return (x_k, F_x_k)
        
    return (x_k, F_x_k)


A = A_task
b = b_task
F = makeF(A, b)
x_min, F_x_min = F4(A, b, F, 3, True)

k = 0: x_k = [1.0, 0.0, 0.0]; F(x_k) = -1.0
k = 1: x_k = [1.0, -0.5, 0.0]; F(x_k) = -1.25
k = 2: x_k = [1.0, -0.5, 0.0]; F(x_k) = -1.25
||F(x_k) - F(x_k+1)|| = 0. exiting


In [100]:
# 2

N = 12

A = np.array([
    [2, 1, 1],
    [1, (3 + 0.1 * N), -1],
    [1, -1, (4 + 0.1 * N)]
])

b = np.array([
    [1],
    [-2],
    [3]
])
_F = makeF(A, b)


# = 2x^2 + (3 + 0.1N)y^2 + (4 + 0.1N)z^2 + xy − yz + xz + x − 2y + 3z + N
F = lambda x: _F(x) + N 

x_grad, F_x_grad = F3(A, b, F, 1000, False)
print(f'Grad: x_grad = {x_grad.T.tolist()[0]}; F(x_grad) = { "{:.7f}".format(F_x_grad) }\n')

x_cord, F_x_cord = F4(A, b, F, 1000, False)
print(f'Cord: x_cord = {x_cord.T.tolist()[0]}; F(x_cord) = {"{:.7f}".format(F_x_cord)}')

Grad: x_grad = [-0.5821003198551427, 0.5281177637904693, -0.36327930854255913]; F(x_grad) = 10.6354032

Cord: x_cord = [-0.5835118183380518, 0.5288066366305914, -0.3630156817367994]; F(x_cord) = 10.6354036
